**5. VISTAS**

In [ ]:
USE bichos_de_bolsillo

GO

-- VISTA PARA VISUALIZAR BICHOS
Create VIEW TablaBichos_V
AS
    SELECT 
        bicho_id AS 'ID', 
        bicho_nombre AS 'Bicho', 
        bicho_salud AS 'Salud',
        (SELECT tipo_nombre FROM Tipos WHERE tipo_id = bicho_tipo) AS 'Tipo',
        (SELECT especie_nombre FROM Especies WHERE especie_id = bicho_especie) AS 'Especie'
    FROM Bichos

GO

-- VISTA PARA VISUALIZAR USUARIOS
Create VIEW UsuariosResgistrados_V
as
    SELECT usuario_nombre as 'Usuarios', usuario_correo AS 'Correo'
    from Usuarios

GO

-- VISTA PARA VISUALIZAR RECOLECCIONES
CREATE VIEW Recolecciones_V
as
    SELECT (SELECT usuario_nombre
        FROM Usuarios
        WHERE usuario_id = recoleccion_usuario) as 'Usuario',
        (SELECT bicho_nombre
        FROM Bichos
        WHERE bicho_id = recoleccion_bicho) AS 'Bicho',
        recoleccion_nivel as 'Nivel',
        recoleccion_fecha as 'Fecha de Recoleccion'
    from Recolecciones

GO

-- VISTA PARA VISUALIZAR COMBATES
CREATE VIEW Combates_V
as
    SELECT
        combate_id as 'ID',
        combate_fecha as 'Fecha',
        (SELECT usuario_nombre
        FROM Usuarios
        WHERE usuario_id = usuario_uno) as 'Jugador 1',
        (SELECT usuario_nombre
        FROM Usuarios
        WHERE usuario_id = usuario_dos) as 'Jugador 2',
        (SELECT count(combate_id)
        FROM Rondas
        WHERE combate_id = Combates.combate_id and ganador = usuario_uno) AS 'Ganadas del jugador 1',
        (SELECT count(combate_id)
        FROM Rondas
        WHERE combate_id = Combates.combate_id and ganador = usuario_dos) AS 'Ganadas del jugador 2'
    FROM Combates

GO

-- VISTA PARA VISUALIZAR BICHOS QUE TIENE UN USUARIO
CREATE VIEW UsuarioBichos_V
AS
    SELECT u.usuario_nombre AS 'Usuario', b.bicho_nombre AS 'Bicho', (SELECT especie_nombre
        FROM Especies
        WHERE especie_id = b.bicho_especie) AS 'Especie', (SELECT tipo_nombre
        FROM Tipos
        WHERE tipo_id = b.bicho_tipo) AS 'Tipo'
    FROM Usuarios u
        INNER JOIN Bichos b ON u.usuario_id = bicho_propietario

GO

-- VISTA PARA VISUALIZAR ATAQUES QUE TIENE UN BICHO
CREATE VIEW BichosAtaque_V
AS
SELECT b.bicho_nombre AS 'Bicho', (SELECT catalogo_nombre
    FROM Catalogo_Ataques
    WHERE catalogo_id = a.ataque_id) AS 'Ataque'
FROM
    Bichos b INNER JOIN Ataques_Bicho a ON b.bicho_id = a.bicho_id

GO


    Probar vistas

In [ ]:
SELECT *
from TablaBichos_V
SELECT *
from UsuariosResgistrados_V
SELECT *
from Recolecciones_V
SELECT *
from Combates_V
SELECT *
FROM UsuarioBichos_V
SELECT *
FROM BichosAtaque_V

**6. FUNCIONES**

In [ ]:
-- FUNCIÓN PARA OBTENER LAS RONDAS DE UN COMBATE
CREATE FUNCTION dbo.GetRondas(@combate_id int)
RETURNS TABLE 
AS 
RETURN 
(
    SELECT
    ronda_noRonda AS 'No. de ronda',
    (SELECT bicho_nombre
    FROM Bichos
    WHERE bicho_id = bicho_uno) AS 'Bicho 1',
    (SELECT bicho_nombre
    FROM Bichos
    WHERE bicho_id = bicho_dos) AS 'Bicho 2',
    (SELECT catalogo_nombre
    FROM Catalogo_Ataques
    WHERE catalogo_id = ataque_uno) AS 'Ataque 1',
    (SELECT catalogo_nombre
    FROM Catalogo_Ataques
    WHERE catalogo_id = ataque_dos) AS 'Ataque 2',
    nivel_daño_uno AS 'Daño 1',
    nivel_daño_dos AS 'Daño 2',
    ganador AS 'Ganador'
FROM Rondas
WHERE combate_id = @combate_id
);

GO

-- FUNCIÓN PARA OBTENER LAS RONDAS DE UN COMBATE QUE GANÓ UN JUGADOR
CREATE FUNCTION dbo.GetRondasPerPlayer(@combate_id int, @usuario int)
RETURNS INT 
AS 
BEGIN
    RETURN (
SELECT count(combate_id)
    FROM Rondas
    WHERE combate_id = @combate_id AND ganador = @usuario
)
END;

GO

-- FUNCIÓN PARA OBTENER EL GANADOR DE UN COMBATE
CREATE FUNCTION dbo.GetGanadorCombate(@combate_id int, @usuario_uno int, @usuario_dos int)
RETURNS INT 
AS 
BEGIN
    DECLARE @ganados1 int, @ganados2 int;

    SET @ganados1 = dbo.GetRondasPerPlayer(@combate_id, @usuario_uno)

    SET @ganados2 = dbo.GetRondasPerPlayer(@combate_id, @usuario_uno)

    IF(@ganados1 > @ganados2)
        RETURN(@ganados1)
    ELSE IF(@ganados1 = @ganados2)
        RETURN(0)
    RETURN(@ganados2)
END;

GO


    Probar funciones

In [ ]:
SELECT *
FROM dbo.GetRondas(1)
SELECT dbo.GetRondasPerPlayer(1, 4) AS 'jugador 1'
SELECT dbo.GetRondasPerPlayer(1, 3) AS 'jugador 2'
SELECT dbo.GetGanadorCombate(1, 4, 3) AS 'Ganador';